# **Selection test Connecthink**

Welcome to the practical selection test for the Data Scientist position at Connectthink. This test is designed to evaluate your skills and knowledge in data analysis, Python programming, use of specific libraries such as pandas, numpy, and your ability to interpret and solve practical data problems.

Please read each section carefully and follow the instructions provided to complete the exercises. The quality of your code, the efficiency of your solutions, and the clarity of your analyzes will be considered in the evaluation. Creativity when answering the questions will also be highly valued. It is important to us that you go beyond what the numbers and graphs tell you. Write the conclusions and results you draw after each part.

Notes:

* You can use ChatGPT or similar to search for documentation, but never copy code directly generated by the model to the notebook.
* Add comments in code to express your reasoning process.
* **Add markdown cells to explain your reasonings and conclusions.**

Good luck!

## **PART 1**: Preparation and Exploratory Analysis of Data

### 1. Load the dataset. how much data do you have how many attributes? is it unbalanced? Are all attributes usable?



### 2. Need to clean up NaNs? why? if yes, do it

### 3. Is it necessary to normalize the data? why? if yes, do it

### 4. Is it necessary to re-encode the data? why? if yes, do it

### 5. Are there strongly correlated variables? if yes, would you perform any operation, combination and/or elimination?

### 6. Which performance measure (F-score, accuracy, etc) is most appropriate for the problem? why?

## **PART 2**: Construction and Evaluation of Classification Models

### 7. Test between 3 and 8 different classification models with different combinations of parameters, applying cross validation (k-fold) to train and evaluate each one. Then, graph the mean and standard deviation of the scores obtained for each model. Based on these results, determine which was the best model. Explain what is the best model.

### For the 2 best classification methods, and justifying the answers with data and/or graphics in addition to a brief explanation:

### 8. What is the best set of parameters for each method

### 9. Which has a better computational cost?

### 10. Regardless of the measure defined in point 6, for each model what would be the best threshold to determine the decision boundary? (You can use the G-mean method)

### 11. Compare confusion matrices and their metrics